In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('spaCy').setMaster('local[*]')
sc = SparkContext(conf=conf)

In [2]:
input_file = 'reviews_dev.json'

In [3]:
with open(input_file, 'r') as f:
    lines = f.readlines()[:10000]

In [4]:
import json
texts = [json.loads(line)['text'] for line in lines]

In [5]:
from adj_noun_extractor import AdjNounExtractor
extractor = AdjNounExtractor()

In [6]:
def extract(text):
    return extractor.extract(text, lemma=True)

def count(pairs):
    res = dict()
    for pair in pairs:
        if pair[1] in res:
            res[pair[1]] += 1
        else:
            res[pair[1]] = 1
    return res

def aggregate(x, y):
    for key in x.keys():
        if key in y:
            y[key] += x[key]
        else:
            y[key] = x[key]
    return y

In [7]:
import time
start = time.time()

rdd = sc.parallelize(texts)
results = rdd.map(extract).map(count).reduce(aggregate)

end = time.time()
print(end - start)

174.4081690311432


In [8]:
import time
start = time.time()

results_2 = dict()
for text in texts:
    pairs = extractor.extract(text, lemma=True)
    for pair in pairs:
        if pair[1] in results_2:
            results_2[pair[1]] += 1
        else:
            results_2[pair[1]] = 1

end = time.time()
print(end - start)

347.26853132247925


In [9]:
results == results_2

True